In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import plotly as px
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler,StandardScaler,LabelEncoder
from sklearn.decomposition import PCA
import sklearn
import xgboost as xgb
from xgboost import XGBRegressor,XGBClassifier
from sklearn.metrics import r2_score,accuracy_score,mean_squared_error as mse
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')


In [2]:
###Project 1 - Mercedes-Benz Greener Manufacturing

#DESCRIPTION

#Reduce the time a Mercedes-Benz spends on the test bench.

# Problem Statement Scenario:
#Since the first automobile, the Benz Patent Motor Car in 1886, Mercedes-Benz has stood for important automotive innovations. These include the passenger safety cell with the crumple zone, the airbag, and intelligent assistance systems. Mercedes-Benz applies for nearly 2000 patents per year, making the brand the European leader among premium carmakers. Mercedes-Benz cars are leaders in the premium car industry. With a huge selection of features and options, customers can choose the customized Mercedes-Benz of their dreams.

#To ensure the safety and reliability of every unique car configuration before they hit the road, Daimler’s engineers have developed a robust testing system. As one of the world’s biggest manufacturers of premium cars, safety and efficiency are paramount on Daimler’s production lines. However, optimizing the speed of their testing system for many possible feature combinations is complex and time-consuming without a powerful algorithmic approach.

#You are required to reduce the time that cars spend on the test bench. Others will work with a dataset representing different permutations of features in a Mercedes-Benz car to predict the time it takes to pass testing. Optimal algorithms will contribute to faster testing, resulting in lower carbon dioxide emissions without reducing Daimler’s standards.

# Following actions should be performed:
# If for any column(s), the variance is equal to zero, then you need to remove those variable(s).
#Check for null and unique values for test and train sets
# Apply label encoder.
# Perform dimensionality reduction.
# Predict your test_df values using xgboost


In [3]:
test=pd.read_csv(r"test.csv")
train=pd.read_csv(r"train.csv")

In [4]:
test.head()

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,az,v,n,f,d,t,a,w,0,...,0,0,0,1,0,0,0,0,0,0
1,2,t,b,ai,a,d,b,g,y,0,...,0,0,1,0,0,0,0,0,0,0
2,3,az,v,as,f,d,a,j,j,0,...,0,0,0,1,0,0,0,0,0,0
3,4,az,l,n,f,d,z,l,n,0,...,0,0,0,1,0,0,0,0,0,0
4,5,w,s,as,c,d,y,i,m,0,...,1,0,0,0,0,0,0,0,0,0


In [5]:
train.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


#FIndings1:Test set has 377 columns and Train set has 378 Columns


In [6]:
#Missing Values in Train set data

train.isna().sum()

ID      0
y       0
X0      0
X1      0
X2      0
       ..
X380    0
X382    0
X383    0
X384    0
X385    0
Length: 378, dtype: int64

In [7]:

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4209 entries, 0 to 4208
Columns: 378 entries, ID to X385
dtypes: float64(1), int64(369), object(8)
memory usage: 12.1+ MB


In [8]:
#Missing values in Test Data
test.isnull().sum()

ID      0
X0      0
X1      0
X2      0
X3      0
       ..
X380    0
X382    0
X383    0
X384    0
X385    0
Length: 377, dtype: int64

In [9]:
test.describe()

,ID,X10,X11,X12,X13,X14,X15,X16,X17,X18,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
count,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,...,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000
mean,4211.039202,0.019007,0.000238,0.074364,0.061060,0.427893,0.000713,0.002613,0.008791,0.010216,...,0.325968,0.049656,0.311951,0.019244,0.011879,0.008078,0.008791,0.000475,0.000713,0.001663
std,2423.078926,0.136565,0.015414,0.262394,0.239468,0.494832,0.026691,0.051061,0.093357,0.100570,...,0.468791,0.217258,0.463345,0.137399,0.108356,0.089524,0.093357,0.021796,0.026691,0.040752
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2115.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4202.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,6310.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,8416.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4209 entries, 0 to 4208
Columns: 377 entries, ID to X385
dtypes: int64(369), object(8)
memory usage: 12.1+ MB


In [11]:
train.dtypes.value_counts()

int64      369
object       8
float64      1
Name: count, dtype: int64

In [12]:
y_train=train['y']

In [13]:
x_train=train.drop('y',axis=1)

In [14]:
x_test=test

In [15]:
#Check Duplicated rows in train dataset
train.duplicated().sum()


0

In [16]:
#Check Duplicated rows in test dataset
test.duplicated().sum()


0

In [17]:
data=train.dtypes

In [18]:
data=pd.DataFrame(data).set_index

In [19]:
data

<bound method DataFrame.set_index of             0
ID      int64
y     float64
X0     object
X1     object
X2     object
...       ...
X380    int64
X382    int64
X383    int64
X384    int64
X385    int64

[378 rows x 1 columns]>

In [20]:
#Find Categorical Data
train_cat=train.select_dtypes(include='object')

In [21]:
#Apply label Encoder
le=LabelEncoder()

In [22]:
for col in train_cat.columns:
   train[col]= le.fit_transform(train[col])
   test[col]=le.fit_transform(train[col])
  

In [23]:
test.head()

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,32,23,17,0,3,24,9,14,0,...,0,0,0,1,0,0,0,0,0,0
1,2,32,21,19,4,3,28,11,14,0,...,0,0,1,0,0,0,0,0,0,0
2,3,20,24,34,2,3,27,9,23,0,...,0,0,0,1,0,0,0,0,0,0
3,4,20,21,34,5,3,27,11,4,0,...,0,0,0,1,0,0,0,0,0,0
4,5,20,23,34,5,3,12,3,13,0,...,1,0,0,0,0,0,0,0,0,0


In [24]:
train.dtypes.value_counts() #To check if the categorical columns are encoded

int64      369
int32        8
float64      1
Name: count, dtype: int64

In [25]:
for column in x_train.columns:
    cardinality = len(np.unique(x_train[column]))
    if cardinality == 1:
        train.drop(column, axis=1) # Column with only one value 
        test.drop(column, axis=1)





In [26]:
# removing features with 0 variance
temp = []
for i in train.columns:
    if train[i].var()==0:
        temp.append(i)
        
print(len(temp))
print(temp)
train.drop(temp,axis=1,inplace=True)
test.drop(temp,axis=1,inplace=True)
print(train.head())
y_train=train['y']
x_drop=train[['y','ID']]
x_train=train.drop(x_drop,axis=1)
x_test=test.drop('ID',axis=1)
id_test = test['ID'].values
x_train.shape,x_test.shape


12
['X11', 'X93', 'X107', 'X233', 'X235', 'X268', 'X289', 'X290', 'X293', 'X297', 'X330', 'X347']
   ID       y  X0  X1  X2  X3  X4  X5  X6  X8  ...  X375  X376  X377  X378  \
0   0  130.81  32  23  17   0   3  24   9  14  ...     0     0     1     0   
1   6   88.53  32  21  19   4   3  28  11  14  ...     1     0     0     0   
2   7   76.26  20  24  34   2   3  27   9  23  ...     0     0     0     0   
3   9   80.62  20  21  34   5   3  27  11   4  ...     0     0     0     0   
4  13   78.02  20  23  34   5   3  12   3  13  ...     0     0     0     0   

   X379  X380  X382  X383  X384  X385  
0     0     0     0     0     0     0  
1     0     0     0     0     0     0  
2     0     0     1     0     0     0  
3     0     0     0     0     0     0  
4     0     0     0     0     0     0  

[5 rows x 366 columns]


((4209, 364), (4209, 364))

In [27]:

#Perform Dimensionality reduction
n_comp = 12
pca = PCA(n_components = n_comp,random_state = 42)
pca2_results_train = pca.fit_transform(x_train)
pca2_results_test = pca.transform(x_test)




In [28]:
#Split Data
x_train,x_val,y_train,y_val = train_test_split(pca2_results_train, y_train, test_size=0.2, random_state=42)

In [29]:
d_train = xgb.DMatrix(x_train,label = y_train)
d_Ytrain = xgb.DMatrix(y_train)
d_val = xgb.DMatrix(x_val,label = y_val)
d_test = xgb.DMatrix(pca2_results_test)
n=50
d_Ytrain 

In [37]:
param={'objective' :'reg:linear','eta':0.2,'max_depth':4}
param1={'n_trees': 500, 
 'eta': 0.0050,
 'max_depth': 3,
 'subsample': 0.95,
 'objective': 'reg:linear',
 'eval_metric': 'rmse',
 'base_score': np.mean(y_train), 
 'silent': 1}

In [57]:
model=XGBRegressor() 
#Prediction with XGB Regressor model
model.fit(x_train,y_train)
pred_val=model.predict(x_val)
r2_score(y_val, pred_val)
mse(y_val, pred_val)

#Prediction with model1 & Model2 hyper parameter
def xgb_r2_score(preds, dtrain):
 labels = dtrain.get_label()
 return 'r2', r2_score(labels, preds)
    
watchlist = [(d_train, 'train'), (d_val, 'valid')]
model1 = xgb.train(param, d_train, 
                1000, watchlist, early_stopping_rounds=50, 
                feval=xgb_r2_score, maximize=True, verbose_eval=10)

model2 = xgb.train(param1, d_train, 
                1000, watchlist, early_stopping_rounds=50, 
                feval=xgb_r2_score, maximize=True, verbose_eval=10)







[0]	train-rmse:11.94617	train-r2:0.11905	valid-rmse:11.67475	valid-r2:0.12432
[10]	train-rmse:9.50939	train-r2:0.44179	valid-rmse:9.50133	valid-r2:0.42001
[20]	train-rmse:8.56345	train-r2:0.54732	valid-rmse:9.17562	valid-r2:0.45909
[30]	train-rmse:8.15433	train-r2:0.58954	valid-rmse:9.05349	valid-r2:0.47340
[40]	train-rmse:7.78220	train-r2:0.62615	valid-rmse:9.03290	valid-r2:0.47579
[50]	train-rmse:7.45484	train-r2:0.65694	valid-rmse:9.04940	valid-r2:0.47387
[60]	train-rmse:7.18613	train-r2:0.68123	valid-rmse:9.09519	valid-r2:0.46853
[70]	train-rmse:6.95245	train-r2:0.70162	valid-rmse:9.08887	valid-r2:0.46927
[80]	train-rmse:6.74786	train-r2:0.71892	valid-rmse:9.09391	valid-r2:0.46868
[90]	train-rmse:6.50821	train-r2:0.73853	valid-rmse:9.12257	valid-r2:0.46533
[91]	train-rmse:6.49448	train-r2:0.73964	valid-rmse:9.12312	valid-r2:0.46527
[0]	train-rmse:12.70980	train-r2:0.00283	valid-rmse:12.45693	valid-r2:0.00305
[10]	train-rmse:12.53752	train-r2:0.02968	valid-rmse:12.27243	valid-r2:0.0

In [60]:
p_test1 = model.predict(pca2_results_test)

In [62]:
sub1 = pd.DataFrame()
sub1['ID'] = id_test
sub1['y'] = p_test1

In [63]:
sub1.head()

,ID,y
0,1,84.625122
1,2,134.913162
2,3,73.294716
3,4,79.613678
4,5,106.861549


In [65]:
p_test2 = model1.predict(d_test)

In [67]:
sub2 = pd.DataFrame()
sub2['ID'] = id_test
sub2['y'] = p_test2

In [68]:
sub2.head()

,ID,y
0,1,81.052353
1,2,106.768951
2,3,80.188606
3,4,77.050278
4,5,111.682846


In [69]:
p_test3 = model2.predict(d_test)
sub3 = pd.DataFrame()
sub3['ID'] = id_test
sub3['y'] = p_test3

In [70]:
sub3.head()

,ID,y
0,1,86.824585
1,2,98.787971
2,3,85.314018
3,4,77.637039
4,5,104.799248


In [ ]:
#Predicted the value using XGBOOSTRegressor(),initialising hyperparameters in XGBoost in sub1,sub2 and sub3 respectively
